In [ ]:
!pwd
! wget https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar -xvzf spark-3.0.1-bin-hadoop3.2.tgz
!pip install findspark

In [300]:
import os
import findspark

os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
findspark.init()

Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [301]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

## Build Spark Session

In [302]:
spark = SparkSession.builder.appName('practical').getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [303]:
df_train = spark.read.csv('train.csv', inferSchema=True, header=True)
df_test = spark.read.csv('test.csv', inferSchema=True, header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [304]:
type(df_train)

pyspark.sql.dataframe.DataFrame

**Show 5 rows.**

In [305]:
df_train.show(5, truncate=False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

**Display schema for the dataset:**

In [306]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [307]:
df_train.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [308]:
df_train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [309]:
df_survived = df_train.filter(df_train['Survived'] == 1)
df_no_survive = df_train.filter(df_train['Survived'] == 0)

**Display your result:**

In [310]:
print('Number of people who survived: ', df_survived.count())
print("Number of people who didn't survive: ", df_no_survive.count())

Number of people who survived:  342
Number of people who didn't survive:  549


**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [311]:
df_train.createOrReplaceTempView("train")

In [312]:
def calc(column):
    return (column/891)
udf_func = f.udf(lambda r: calc(r))

In [313]:
ratio = spark.sql(""" SELECT Survived,COUNT('Survived') FROM train GROUP BY Survived""")
ratio.withColumn("Ratio", udf_func(f.col("count(Survived)"))) \
  .show(truncate=False)

+--------+---------------+------------------+
|Survived|count(Survived)|Ratio             |
+--------+---------------+------------------+
|1       |342            |0.3838383838383838|
|0       |549            |0.6161616161616161|
+--------+---------------+------------------+



**Can you get the number of males and females?**


In [314]:
female_count = df_train.filter(df_train['Sex'] == 'female').count()
male_count = df_train.filter(df_train['Sex'] == 'male').count()

print('Males in dataset: ', male_count)
print('Females in dataset: ', female_count)

Males in dataset:  577
Females in dataset:  314


**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [315]:
df_temp_survive = df_train.where('Survived = 1').groupBy('Sex').count()
df_temp_survive.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



In [316]:
print('avg female survival: ', df_temp_survive.select('count').where("sex = 'female'").collect()[0][0] / female_count)

print('avg male survival: ',df_temp_survive.select('count').where("sex = 'male'").collect()[0][0] / male_count)

avg female survival:  0.7420382165605095
avg male survival:  0.18890814558058924


**Create temporary view PySpark:**

In [317]:
df_train.createOrReplaceTempView('train_view')

**How many people survived, and how many didn't survive? By SQL:**

In [318]:
spark.sql('select count(*) as survival_count from train_view where survived = 1').show()

+--------------+
|survival_count|
+--------------+
|           342|
+--------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [319]:
spark.sql('select sex, round(avg(survived),2) as survival from train_view group by sex').show()

+------+--------+
|   sex|survival|
+------+--------+
|female|    0.74|
|  male|    0.19|
+------+--------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [320]:
spark.sql('select pclass, round(sum(survived)/ count(*), 2) as avg_survival from train_view group by pclass').show()

+------+------------+
|pclass|avg_survival|
+------+------------+
|     1|        0.63|
|     3|        0.24|
|     2|        0.47|
+------+------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [321]:
df = df_train.union(df_test)
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

**Display count:**

In [322]:
df.count()

1329

**Can you define the number of null values in each column?**


In [323]:
temp_df = df.select([f.count(f.when(f.isnan(col) | f.col(col).isNull(), col)).alias(col) for col in df.columns])
temp_df.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [324]:
nulls_df = spark.createDataFrame([[column, v] for column,v in zip(temp_df.columns, temp_df.head())], schema=["Name", "Null Counts"])
nulls_df.show()

+-----------+-----------+
|       Name|Null Counts|
+-----------+-----------+
|PassengerId|          0|
|   Survived|          0|
|     Pclass|          0|
|       Name|          0|
|        Sex|          0|
|        Age|        265|
|      SibSp|          0|
|      Parch|          0|
|     Ticket|          0|
|       Fare|          0|
|      Cabin|       1021|
|   Embarked|          3|
+-----------+-----------+



## Preprocessing 

**Create Temporary view PySpark:**

In [325]:
df.createOrReplaceTempView('data_view')

**Can you show the "name" column from your temporary table?**

In [326]:
spark.sql('select name from data_view').show()

+--------------------+
|                name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [327]:
import pyspark.sql.functions as F
combined = df.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display "Title" column and count "Title" column:**

In [328]:
title_df = spark.sql('select title, count(title) as total_count from combined group by title')
title_df.show()

+--------+-----------+
|   title|total_count|
+--------+-----------+
|     Don|          1|
|    Miss|        257|
|Countess|          2|
|     Col|          4|
|     Rev|          9|
|    Lady|          2|
|  Master|         56|
|     Mme|          1|
|    Capt|          2|
|      Mr|        786|
|      Dr|         11|
|     Mrs|        186|
|     Sir|          2|
|Jonkheer|          2|
|    Mlle|          4|
|   Major|          3|
|      Ms|          1|
+--------+-----------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [329]:
rare_titles_dict = dict(title_df.rdd.map(lambda x: (x['title'], "rare" if x['total_count'] < 10 else x['title'])).collect())

In [330]:
rare_titles_dict

{'Capt': 'rare',
 'Col': 'rare',
 'Countess': 'rare',
 'Don': 'rare',
 'Dr': 'Dr',
 'Jonkheer': 'rare',
 'Lady': 'rare',
 'Major': 'rare',
 'Master': 'Master',
 'Miss': 'Miss',
 'Mlle': 'rare',
 'Mme': 'rare',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Ms': 'rare',
 'Rev': 'rare',
 'Sir': 'rare'}

**Run the function:**

In [331]:
def impute_title(title):
    return rare_titles_dict[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [332]:
title_udf = f.udf(lambda title : impute_title(title))

**Display "Title" from table and group by "Title" column:**

In [333]:
combined  = combined.withColumn('Title', title_udf(f.col('Title')))

In [334]:
combined.groupBy('Title').agg(f.count('Title')).show()

+------+------------+
| Title|count(Title)|
+------+------------+
|  rare|          33|
|  Miss|         257|
|Master|          56|
|    Mr|         786|
|    Dr|          11|
|   Mrs|         186|
+------+------------+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [335]:
age_mean = round(combined.agg(f.avg('Age')).collect()[0][0],2)

**Fill missing with "age" mean:**

In [336]:
combined = combined.na.fill(age_mean,['Age'])
combined.show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35.0|    0|    0|          373450|   8.05| null|       S|    Mr|
|          6|   

## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [337]:
x = spark.sql('select Embarked, count(*) as count from combined group by Embarked order by count desc')

**Show "groupped_Embarked" your variable:**

In [338]:
x.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get max of groupped_Embarked:** 

In [339]:
max_value = x.collect()[0][0]

**Fill missing values with max 'S' of grouped_Embarked:**

In [340]:
combined = combined.na.fill(max_value,['Embarked'])

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [341]:
def replace_with_first_char(cab):
  return cab[0]

combined = combined.withColumn('Cabin', f.substring('Cabin', 0, 1))

**Show the result:**

In [342]:
combined.show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38.0|    1|    0|        PC 17599|71.2833|    C|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35.0|    1|    0|          113803|   53.1|    C|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35.0|    0|    0|          373450|   8.05| null|       S|    Mr|
|          6|   

**Create the temporary view:**

In [343]:
combined.createOrReplaceTempView('comb_view')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [344]:
spark.sql("select cabin, count(cabin) as total from comb_view group by cabin order by total desc").show()

+-----+-----+
|cabin|total|
+-----+-----+
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
| null|    0|
+-----+-----+



**Fill missing values with "U":**

In [345]:
combined = combined.na.fill('U', ['Cabin'])

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [346]:
import pyspark.ml.feature as ml

In [347]:
categorical_cols = [field for (field, dataType) in combined.dtypes
                   if dataType == "string" and field not in ['Ticket', 'Name']]

In [348]:
categorical_cols

['Sex', 'Cabin', 'Embarked', 'Title']

In [349]:
index_output_cols = [x + "_Index" for x in categorical_cols]
index_output_cols

['Sex_Index', 'Cabin_Index', 'Embarked_Index', 'Title_Index']

In [350]:
ohe_output_cols = [x + "_OHE" for x in categorical_cols]
ohe_output_cols

['Sex_OHE', 'Cabin_OHE', 'Embarked_OHE', 'Title_OHE']

In [351]:
string_indexer = ml.StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols)

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [352]:
one_encoder = ml.OneHotEncoder(inputCols=index_output_cols, outputCols=ohe_output_cols)

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [353]:
numerical_cols =[field for (field,dataType) in combined.dtypes
              if ((dataType!='string')& (field not in ['Survived', 'PassengerId']))]
numerical_cols

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [354]:
mixed_features = ohe_output_cols + numerical_cols
mixed_features

['Sex_OHE',
 'Cabin_OHE',
 'Embarked_OHE',
 'Title_OHE',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

In [355]:
vector_assembler = ml.VectorAssembler(inputCols=mixed_features, outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [356]:
train_df, test_df = combined.randomSplit([.8,.2],seed=42)
print(f"There are {train_df.count()} rows in the training set, and {test_df.count()} in the test set")

There are 1094 rows in the training set, and 235 in the test set


**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [357]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='Survived')
pipeline = Pipeline(stages=[string_indexer, one_encoder, vector_assembler, rf])

In [358]:
model=pipeline.fit(train_df)

In [359]:
pred = model.transform(test_df)

In [360]:
pred.select('features','Survived','prediction').show(5)

+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|(21,[1,9,12,16,17...|       1|       1.0|
|(21,[0,5,9,11,16,...|       0|       0.0|
|(21,[1,9,13,16,17...|       1|       1.0|
|(21,[0,1,9,11,16,...|       0|       0.0|
|(21,[1,10,13,16,1...|       1|       1.0|
+--------------------+--------+----------+
only showing top 5 rows



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [361]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Survived', metricName='accuracy')

In [362]:
eval.evaluate(pred)

0.8212765957446808

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
